#
---
- cifar10

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [63]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [64]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=True)

X_train.shape, X_val.shape, X_test.shape

((35000, 32, 32, 3), (15000, 32, 32, 3), (10000, 32, 32, 3))

## 정규화

In [65]:
X_train = X_train.astype('float32')/255.
X_val = X_val.astype('float32')/255.
X_test = X_test.astype('float32')/255.

In [66]:
# y -> onehot 안하는 방법
from tensorflow.keras.utils import to_categorical

y_train_o = to_categorical(y_train)
y_val_o = to_categorical(y_val)
y_test_o = to_categorical(y_test)

In [67]:
y_test.shape, y_train_o.shape

((10000, 1), (35000, 10))

## 모델

In [68]:
from tensorflow.keras.applications import VGG16

base = VGG16(weights = 'imagenet', 
             input_shape = (32,32,3),
             include_top = False  # top = flatten 이후부분은 떼어버릴 것. filter부분만 사용한다는 뜻
             )
base.trainable=False # weight가 더이상 학습하지 못하게 막음

base.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

- 출력이 1000으로 되어있음. 이걸 바꿀것임
- 위에서 include_top 설정을 바꾼 결과, flatten 부분부터 떼어졌다

In [79]:
model = keras.Sequential([
    base,
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer = 'adam',
    loss= 'categorical_crossentropy',
    metrics = ['acc']
)

In [70]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_12 (Flatten)        (None, 512)               0         
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 dense_24 (Dense)            (None, 64)                32832     
                                                                 
 dense_25 (Dense)            (None, 1)                 65        
                                                                 
Total params: 14,747,585
Trainable params: 32,897
Non-trainable params: 14,714,688
_________________________________________________________________


In [71]:
from tensorflow.keras import callbacks

callpath = 'call.ckpt'
callpoint = callbacks.ModelCheckpoint(
    callpath,
    monitor = 'val_loss',
    save_weights_only = True,
    save_best_only = True,
    verbose = 1
)

In [72]:
es_callback = callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5
)

## 학습

In [78]:
y_train_o.shape

(35000, 10)

In [81]:
EPOCHS = 100
BATCH_SIZE = 64

log = model.fit(X_train, y_train_o,
                epochs = EPOCHS,
                batch_size = BATCH_SIZE,
                validation_data = (X_val,y_val_o),
                callbacks = [callpoint, es_callback],
                verbose = 1)

Epoch 1/100
544/547 [============================>.] - ETA: 0s - loss: 1.5644 - acc: 0.4530

ValueError: ignored

In [74]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure(figsize=(16,8))
    plt.subplot(1,2,1)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(hist['epoch'], hist['loss'],label='Train Loss')
    plt.plot(hist['epoch'], hist['val_loss'],label = 'Val Loss')
    plt.legend()

    plt.subplot(1,2,2)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(hist['epoch'], hist['acc'],label='Train Accuracy')
    plt.plot(hist['epoch'], hist['val_acc'],label = 'Val Accuracy')

    plt.legend()
    plt.show()

In [ ]:
plot_history(log)

In [82]:
model.evaluation(X_test, y_test_o)

AttributeError: ignored